In [18]:
# %matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, Angle
from astropy.time import Time
from astropy import units as u
import tqdm
import random
import string
from astropy.visualization import astropy_mpl_style, quantity_support

In this noteboook I keep the old codes that is necesary to rule out for different reasons, or for try different options with other codes

In [ ]:
def Order(Data,rango,time,General_time,K):
    targets = []
    order = [] 

    if Data.empty:
        return 'Theres no targets for observations'

    for i in range(1,rango):
        #Organize and add time to the dataframe
        Target = Data.loc[Data['Label'] == i].copy()

        #If theres no dataframe continue
        if Target.empty:
            continue
        
        Target['Time'] = time

        #Calculate the time exposition
        Target['Time expo'] = ExpositionTime(K, Target['Mag'])

        #Do not take the targets that are not observable and the time expo is in the slot of (1-3600)s
        Target = Target[(Target['Observable'] != False) & (Target['Time expo'].between(1, 3600, inclusive='neither'))]

        #Take from dataframe just the observables
        if not Target.empty:
            targets.append(Target)

    if not targets:
        return 'Theres no targets for observations'

    #Concate everything for a better analysis
    obs = pd.concat(targets,axis=0).reset_index(drop=True).drop(['Observable'],axis=1)

    #Determine the flag variables for the cycle, the first position is gonna be always the first point
    first_target = obs[obs['Time'] == time[0]]
    if not first_target.empty:
        first_target = first_target.sort_values(by='Alt', ascending=False, na_position='first').head(1)
        first_target['Time'] = General_time[0]
        order.append(first_target)
        
        time_flag = General_time[0] + int(first_target['Time expo'].iloc[0]) * u.second
        position_flag = first_target
    else:
        time_flag = General_time[0]
        position_flag = pd.DataFrame()


    #Stablish the order of observation by highest altitude and the time exposition
    for i in range(1,len(time)):
        #Organice by highest altitude
        priority = obs.loc[obs['Time'] == time[i]].sort_values('Alt', ascending=False, na_position='first').head(1)
        current_time = General_time[i]

        #No matter if we have an empty entrace
        if priority.empty:
            continue

        #Give the time for the observable
        priority['Time'] = current_time

        #No take the empty dataframes
        if  current_time <= time_flag:
            
            if not position_flag.empty and (position_flag['Label'].iloc[0] == priority['Label'].iloc[0]):
                order.append(priority)
                time_flag = current_time
            else:
                position_flag['Time'] = current_time
                order.append(position_flag)
                time_flag = current_time

        if  current_time > time_flag:
            if not position_flag.empty and (position_flag['Label'].iloc[0] == priority['Label'].iloc[0]):
                order.append(priority)
                time_flag = current_time
            else:
                order.append(position_flag)
                time_flag = current_time + int(priority['Time expo'].iloc[0]) * u.second
                position_flag = priority 
            
    #Take just the dataframe necessary 
    order = pd.concat(order,axis=0).reset_index(drop=True).drop(['Az','Alt'],axis=1)

    if order.empty:
        return 'No hay observaciones'
    return order

Order(Data,many,time,general_time,K)

In [ ]:
#Made the delta time for the graphic
def DeltaTime(Date_i,Date_f,t_scale):

    scale = t_scale[0]
    sc = t_scale[1]

    a = Time(Date_i,format = 'iso', scale='utc').datetime.hour-24
    b = Time(Date_f,format = 'iso', scale='utc').datetime.hour

    if scale == 's':
        len = int((b-a)/ (sc/3600))
        t_ = np.linspace(a , b , len) 
   
    if scale == 'm':
        len = int((b-a)/ (sc/60))
        t_ = np.linspace(a , b , len)
    
    if scale == 'h':
        len = int((b-a)/ sc)
        t_ = np.linspace(a , b , len)
    
    return t_

#Made the array of time around of midnight
def CreateTime(Date_i,Date_f,t_scale):

    time_midnight = Time( Time(Date_i,format = 'iso', scale='utc').iso.split()[0] + ' 00:00:00' ,format = 'iso', scale='utc')

    delta = DeltaTime(Date_i,Date_f,t_scale)*u.hour

    return time_midnight + delta

#Return the classified observations
def Observations(observer, alert, Date_i, Date_f,time_scale,rango,limit,m_min):

    #Definition of different variables:
    time = CreateTime(Date_i,Date_f,time_scale) #return a array

    lat_conv , lon_conv = observer
    observer = EarthLocation(lat=lat_conv*u.deg, lon=lon_conv*u.deg)

    #Lists of data for post-analysis
    Big_Data = []

    #Give the label of identification of each object
    alert['Label'] = range(1, rango)

    #Give the conditional for the minimum observation
    alert['V mag'] = alert['Mag'] <= m_min

    #Get off which can not be visible and Quit that column because its just an indicator
    alert = alert[alert['V mag'] != False].drop(['V mag'],axis=1)

    #Establish the time slot
    for each_time in time:

        #For each time slot we need the data of the conditional about observations
        
        #Establish the coordinates with the correction from geocentric calculations of astropy
        celestial_coord = SkyCoord(ra=alert['RA'], dec=alert['DEC']) #Mantain the degrees units

        # Calculate the coordenates AltAz for the time and observer and transform the J2000 coordinate system for altaz
        altaz_coord = celestial_coord.transform_to( AltAz(obstime=each_time, location=observer) ) 
    
        # Determinate if its observable and a good observable (altitude > 33 degrees) in the altaz coordinate system, return a boolean.
        state  = altaz_coord.alt > limit*u.deg

        #Put the things together
        alert['Observable'] = state
        alert['Az'] = altaz_coord.az.deg
        alert['Alt'] = altaz_coord.alt.deg
        
        #Put the conditionals of a time slot on a dataframe in the format for better reading
        Data = alert.copy()
        
        #Put the dataframe in a list and the values of time
        Big_Data.append(Data) 

    #With the DataFrame we can organice all the observations
    final_data = pd.concat(Big_Data,axis=0)

    return final_data


#Calculate exposition time
def ExpositionTime(K, m, m_ref):
    return K*(10**(0.4*(m - m_ref)))


#Return the dataframe with the information about graphic and order
def OrderLimits(Data,rango,Time,General_time,t_expo,K,m_ref):
    targets = []
    order = [] 

    for i in range(1,rango):
        #Organize and add time to the dataframe
        Target = Data.loc[Data['Label'] == i].copy()
        if Target.empty:
            continue
        
        Target['Time'] = Time
        Target['Time expo'] = ExpositionTime(K, Target['Mag'], m_ref)
        Target = Target[(Target['Observable'] != False) & (Target['Time expo'] <= t_expo)]

        #Take from dataframe just the observables
        if not Target.empty:
            targets.append(Target)

    if not targets:
        return [], pd.DataFrame()

    #Concate everything for a better analysis
    obs = pd.concat(targets,axis=0).reset_index(drop=True).drop(['Observable'],axis=1)


    #Determine the flag variables for the cycle, the first position is gonna be always the first point
    first_target = obs[obs['Time'] == Time[0]]
    if not first_target.empty:
        first_target = first_target.sort_values(by='Alt', ascending=False, na_position='first').head(1)
        first_target['Time'] = General_time[0]
        order.append(first_target)
        
        time_flag = General_time[0] + first_target['Time expo'].iloc[0] * u.second
        position_flag = first_target
    else:
        time_flag = General_time[0]
        position_flag = pd.DataFrame()


    #Stablish the order of observation by highest altitude and the time exposition
    for i in range(1,len(Time)):
        #Organice by highest altitude
        priority = obs.loc[obs['Time'] == Time[i]].sort_values('Alt', ascending=False, na_position='first').head(1)
        current_time = General_time[i]

        #No matter if we have an empty entrace
        if priority.empty:
            continue

        #Give the time for the observable
        priority['Time'] = current_time

        #No take the empty dataframes
        if priority.empty==False:

            if time_flag >= current_time:

                if not position_flag.empty and position_flag['Label'].iloc[0] == priority['Label'].iloc[0]:
                    order.append(priority)
                else:
                    position_flag['Time'] = current_time
                    order.append(position_flag)
            else:
                order.append(priority)
                time_flag = current_time + priority['Time expo'].iloc[0] * u.second
                position_flag = priority

    #Take just the dataframe necessary
    order = pd.concat(order,axis=0).reset_index(drop=True).drop(['Az','Alt'],axis=1)

    return order


In [ ]:
def Order(Data,rango,time,General_time,K):
    targets = []
    order = [] 

    if Data.empty:
        return 'Theres no targets for observations'

    for i in range(1,rango):
        #Organize and add time to the dataframe
        Target = Data.loc[Data['Label'] == i].copy()

        #If theres no dataframe continue
        if Target.empty:
            continue
        
        Target['Time'] = time

        #Calculate the time exposition
        Target['Time expo'] = ExpositionTime(K, Target['Mag'])

        #Do not take the targets that are not observable and the time expo is in the slot of (1-3600)s
        Target = Target[(Target['Observable'] != False) & (Target['Time expo'].between(1, 3600, inclusive='neither'))]

        #Take from dataframe just the observables
        if not Target.empty:
            targets.append(Target)

    if not targets:
        return 'Theres no targets for observations'

    #Concate everything for a better analysis
    obs = pd.concat(targets,axis=0).reset_index(drop=True).drop(['Observable'],axis=1)

    #Determine the flag variables for the cycle, the first position is gonna be always the first point
    first_target = obs[obs['Time'] == time[0]]
    if not first_target.empty:
        first_target = first_target.sort_values(by='Alt', ascending=False, na_position='first').head(1)
        first_target['Time'] = General_time[0]
        order.append(first_target)
        
        time_flag = General_time[0] + int(first_target['Time expo'].iloc[0]) * u.second
        position_flag = first_target
    else:
        time_flag = General_time[0]
        position_flag = pd.DataFrame()


    #Stablish the order of observation by highest altitude and the time exposition
    for i in range(1,len(time)):
        #Organice by highest altitude
        priority = obs.loc[obs['Time'] == time[i]].sort_values('Alt', ascending=False, na_position='first').head(1)
        current_time = General_time[i]

        #No matter if we have an empty entrace
        if priority.empty:
            continue

        #Give the time for the observable
        priority['Time'] = current_time

        #No take the empty dataframes
        if  current_time <= time_flag:
            
            if not position_flag.empty and (position_flag['Label'].iloc[0] == priority['Label'].iloc[0]):
                order.append(priority)
                time_flag = current_time
            else:
                position_flag['Time'] = current_time
                order.append(position_flag)
                time_flag = current_time

        if  current_time > time_flag:
            if not position_flag.empty and (position_flag['Label'].iloc[0] == priority['Label'].iloc[0]):
                order.append(priority)
                time_flag = current_time
            else:
                order.append(position_flag)
                time_flag = current_time + int(priority['Time expo'].iloc[0]) * u.second
                position_flag = priority 
            
    #Take just the dataframe necessary 
    order = pd.concat(order,axis=0).reset_index(drop=True).drop(['Az','Alt'],axis=1)

    if order.empty:
        return 'No hay observaciones'
    return order

Order(Data,many,time,general_time,K)

In [ ]:

#Tuple in order of (latitude,longitude) in degrees
observer = 4.609 , -74.081
 
#Slot of time and date
date_i = '2025-03-13 19:00:00'
date_f = '2025-03-14 07:00:00'

#Time scale of the observation, the minimum is 5 seconds
timescale = ['m', 30]

#Limit of telescopes for good observations, usually are 33 degree on altitude
limit = 33 

# Limit on magnitude for the specific telescope
m_min = 30

# Parameter K and m_ref, random numbers until i define the units
K = 10
m_ref = 10 

#Minimum time for exposition that have the telescope in seconds
t_expo = 30

alerts = [{'Name': ['ATKFRLD', 'EFJGODC', 'FQECBTV'],
'RA': ['09h21m58.31s','17h06m56.28s','11h44m44.33s'],
'DEC': ['+24d39m47.46s','+86d58m54.09s','+74d02m42.41s'],
'Mag': [-7.59,1.16,-2.66]
}]

many = len(alerts) + 1

In [31]:
#Dataframe with random targets
alerts = random_data(5)

#Amount of targets, this is for the labels of graphic
many = len(alerts) + 1

print(alerts)

      Name            RA            DEC    Mag
0  ATKFRLD  09h21m58.31s  +24d39m47.46s  -7.59
1  EFJGODC  17h06m56.28s  +86d58m54.09s   1.16
2  FQECBTV  11h44m44.33s  +74d02m42.41s  -2.66
3   XERBSU  12h59m19.94s  +55d53m27.18s  16.44
4  LYWARER  15h13m19.06s  -30d06m02.53s   7.05
